In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [42]:
df = pd.read_csv('preprocessed_data.csv')

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38050 entries, 0 to 38049
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    38050 non-null  int64  
 1   clean_text    38050 non-null  object 
 2   category      38050 non-null  float64
 3   Length        38050 non-null  float64
 4   Preprocessed  38050 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 1.5+ MB


In [44]:
# Preprocess the data
X = df['Preprocessed']
y = df['category']

In [45]:
# Tokenize the text and train Word2Vec embeddings
vector_size = 100
window = 5
min_count = 1
sg = 1  # Skip-gram (use sg=0 for CBOW)

In [46]:
tokenized_texts = [text.split() for text in X]
word2vec_model = Word2Vec(tokenized_texts, vector_size=vector_size, window=window, min_count=min_count, sg=sg)


In [47]:
# Create average word embeddings for each text
X_embeddings = np.array([
    np.mean([word2vec_model.wv[word] for word in text if word in word2vec_model.wv], axis=0)
    for text in tokenized_texts
])


In [48]:
# Split into train and validation sets
X_train_embeddings, X_val_embeddings, y_train, y_val = train_test_split(X_embeddings, y, test_size=0.3, random_state=42)


In [49]:
# Multinomial NB

# Fitting Naive Bayes to the Training set

clf = Pipeline([
    ('Normalizing',MinMaxScaler()),
     ('MultinomialNB',MultinomialNB())
     ])

clf.fit(X_train_embeddings,y_train)


# Predicting the Test set results
y_pred = clf.predict(X_val_embeddings)

In [50]:
print(classification_report(y_val,y_pred))


              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00      3832
         1.0       0.66      1.00      0.80      7583

    accuracy                           0.66     11415
   macro avg       0.33      0.50      0.40     11415
weighted avg       0.44      0.66      0.53     11415



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
